In [ ]:
!pip install scikit-learn-extra


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 18.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-learn-extra: filename=scikit_learn_extra-0.3.0-cp312-cp312-linux_x86_64.whl size=1959513 sha256=2e4288262ef8cd482d19ef5d8df0f9f7b4f7b5ced895c702e6b027cd7aa45753
  Stored in directory: /root/.cache/pip/wheels/17/4d/c3/c6d5d563c1bf8146d059d63be3678abc2f2801fba0aaf5f0b8
Successfully built scikit-learn-extra


In [ ]:
!pip install hdbscan


In [ ]:
!pip install scikit-fuzzy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 20.4 MB/s eta 0:00:00


In [ ]:
!pip uninstall -y numpy


Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2


In [ ]:
!pip install "numpy<2"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 71.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.13.0.90 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.37.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
rasterio 1.5.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.13.0.90 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.13.0.90 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
tobler 0.13.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have n

In [ ]:
import numpy as np
import os
from sklearn.cluster import (
    KMeans,
    MiniBatchKMeans,
    AgglomerativeClustering,
    SpectralClustering,
    MeanShift,
    AffinityPropagation,
    DBSCAN,
    OPTICS,
    Birch,
    BisectingKMeans
)

from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.preprocessing import normalize

from sklearn_extra.cluster import KMedoids
import hdbscan
import skfuzzy as fuzz


/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
original_df = pd.read_csv("original_labels.csv")

image_names = original_df["image_name"].values
original_labels = original_df["label"].values

le = LabelEncoder()
original_labels_enc = le.fit_transform(original_labels)

In [ ]:
def compute_label_change(original, clustered):
    return (np.sum(original != clustered) / len(original)) * 100


def save_cluster_labels_csv(model, algo, image_names, original_labels, labels):
    os.makedirs("clustering_labels", exist_ok=True)

    df = pd.DataFrame({
        "image_name": image_names,
        "original_label": original_labels,   # ← human-readable
        "cluster_label": labels               # ← cluster assignment
    })

    filename = f"clustering_labels/{model}_{algo}.csv"
    df.to_csv(filename, index=False)



In [ ]:
def get_algorithms(X):
    """
    Returns clustering algorithms that do NOT require training images.
    """

    X_norm = normalize(X)

    return {

        # # ================= PARTITION-BASED =================
        "KMeans": KMeans(n_clusters=NUM_CLASSES, random_state=42),

        "MiniBatchKMeans": MiniBatchKMeans(
            n_clusters=NUM_CLASSES, random_state=42
        ),

        "KMedoids_PAM": KMedoids(
            n_clusters=NUM_CLASSES, method="pam", random_state=42
        ),

        "BisectingKMeans": BisectingKMeans(
            n_clusters=NUM_CLASSES, random_state=42
        ),

        # Spherical K-Means (handled as callable)
        "SphericalKMeans": lambda X: KMeans(
            n_clusters=NUM_CLASSES,
            random_state=42
        ).fit_predict(X_norm),

        # Fuzzy C-Means (callable)
        "FuzzyCMeans": lambda X: np.argmax(
            fuzz.cluster.cmeans(
                X_norm.T,
                c=NUM_CLASSES,
                m=2.0,
                error=0.005,
                maxiter=1000
            )[1],
            axis=0
        ),

        # ================= HIERARCHICAL =================
        "Agglomerative_Single": AgglomerativeClustering(
            n_clusters=NUM_CLASSES, linkage="single"
        ),

        "Agglomerative_Complete": AgglomerativeClustering(
            n_clusters=NUM_CLASSES, linkage="complete"
        ),

        "Agglomerative_Average": AgglomerativeClustering(
            n_clusters=NUM_CLASSES, linkage="average"
        ),

        "Agglomerative_Ward": AgglomerativeClustering(
            n_clusters=NUM_CLASSES, linkage="ward"
        ),

        # ================= DENSITY-BASED =================
        "DBSCAN": DBSCAN(eps=0.5, min_samples=5),

        "OPTICS": OPTICS(min_samples=5),

        "HDBSCAN": hdbscan.HDBSCAN(min_cluster_size=10),

        "MeanShift": MeanShift(),

        # ================= MODEL-BASED =================
        "GMM": GaussianMixture(
            n_components=NUM_CLASSES,
            reg_covar=1e-4,
            random_state=42
        ),

        "BayesianGMM": BayesianGaussianMixture(
            n_components=NUM_CLASSES,
            random_state=42
        ),

        # # ================= GRAPH-BASED =================
        # "SpectralClustering": SpectralClustering(
        #     n_clusters=NUM_CLASSES,
        #     assign_labels="kmeans",
        #     random_state=42
        # ),

        # ================= LARGE-SCALE =================
        "BIRCH": Birch(n_clusters=NUM_CLASSES),

        # ================= MESSAGE PASSING =================
        "AffinityPropagation": AffinityPropagation()
    }


In [ ]:
NUM_CLASSES = 3
MODEL_NAME = "EfficientNetB0"

print(f"\n===== Processing {MODEL_NAME} =====")

X = np.load(f"{MODEL_NAME}_features.npy").astype(np.float64)

algorithms = get_algorithms(X)
results = []

for algo_name, algo in algorithms.items():
    print(f"→ Implementing {algo_name} ...")

    try:
        # Case 1: callable algorithms (Fuzzy, Spherical)
        if callable(algo):
            labels = algo(X)

        # Case 2: GMM / BayesianGMM
        elif hasattr(algo, "predict") and not hasattr(algo, "fit_predict"):
            algo.fit(X)
            labels = algo.predict(X)

        # Case 3: sklearn clustering
        else:
            labels = algo.fit_predict(X)

        # Skip degenerate clustering
        if len(np.unique(labels)) < 2:
            raise ValueError("Only one cluster formed")

        save_cluster_labels_csv(
            MODEL_NAME,
            algo_name,
            image_names,
            original_labels,
            labels
        )

        change = compute_label_change(original_labels_enc, labels)
        results.append([MODEL_NAME, algo_name, change])

        print(f"   ✓ Done | Label change: {change:.2f}%")

    except Exception as e:
        print(f"   ✗ Failed: {str(e)}")
        continue


pd.DataFrame(
    results,
    columns=["model", "clustering", "label_percentage_change"]
).to_csv(
    "clustering_results.csv",
    mode="a",
    header=not os.path.exists("clustering_results.csv"),
    index=False
)



===== Processing EfficientNetB0 =====
→ Implementing KMeans ...
   ✓ Done | Label change: 92.67%
→ Implementing MiniBatchKMeans ...
   ✓ Done | Label change: 48.78%
→ Implementing KMedoids_PAM ...
   ✓ Done | Label change: 63.41%
→ Implementing BisectingKMeans ...
   ✓ Done | Label change: 44.03%
→ Implementing SphericalKMeans ...
   ✓ Done | Label change: 94.11%
→ Implementing FuzzyCMeans ...
   ✓ Done | Label change: 42.79%
→ Implementing Agglomerative_Single ...
   ✓ Done | Label change: 68.94%
→ Implementing Agglomerative_Complete ...
   ✓ Done | Label change: 66.83%
→ Implementing Agglomerative_Average ...
   ✓ Done | Label change: 66.35%
→ Implementing Agglomerative_Ward ...
   ✓ Done | Label change: 89.65%
→ Implementing DBSCAN ...
   ✓ Done | Label change: 54.95%
→ Implementing OPTICS ...
   ✓ Done | Label change: 99.90%
→ Implementing HDBSCAN ...
   ✓ Done | Label change: 87.62%
→ Implementing MeanShift ...
   ✓ Done | Label change: 92.67%
→ Implementing GMM ...
   ✓ Done | L

/usr/local/lib/python3.12/dist-packages/sklearn/cluster/_affinity_propagation.py:140: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


   ✓ Done | Label change: 99.98%


In [ ]:
NUM_CLASSES = 3
MODEL_NAME = "ResNet152"

print(f"\n===== Processing {MODEL_NAME} =====")

X = np.load(f"{MODEL_NAME}_features.npy").astype(np.float64)

algorithms = get_algorithms(X)
results = []

for algo_name, algo in algorithms.items():
    print(f"→ Implementing {algo_name} ...")

    try:
        # Case 1: callable algorithms (Fuzzy, Spherical)
        if callable(algo):
            labels = algo(X)

        # Case 2: GMM / BayesianGMM
        elif hasattr(algo, "predict") and not hasattr(algo, "fit_predict"):
            algo.fit(X)
            labels = algo.predict(X)

        # Case 3: sklearn clustering
        else:
            labels = algo.fit_predict(X)

        # Skip degenerate clustering
        if len(np.unique(labels)) < 2:
            raise ValueError("Only one cluster formed")

        save_cluster_labels_csv(
            MODEL_NAME,
            algo_name,
            image_names,
            original_labels,
            labels
        )

        change = compute_label_change(original_labels_enc, labels)
        results.append([MODEL_NAME, algo_name, change])

        print(f"   ✓ Done | Label change: {change:.2f}%")

    except Exception as e:
        print(f"   ✗ Failed: {str(e)}")
        continue


pd.DataFrame(
    results,
    columns=["model", "clustering", "label_percentage_change"]
).to_csv(
    "clustering_results.csv",
    mode="a",
    header=not os.path.exists("clustering_results.csv"),
    index=False
)



===== Processing ResNet152 =====
→ Implementing KMeans ...
   ✗ Failed: Only one cluster formed
→ Implementing MiniBatchKMeans ...
   ✗ Failed: Only one cluster formed
→ Implementing KMedoids_PAM ...


/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


   ✗ Failed: Only one cluster formed
→ Implementing BisectingKMeans ...
   ✗ Failed: Only one cluster formed
→ Implementing SphericalKMeans ...
   ✗ Failed: Only one cluster formed
→ Implementing FuzzyCMeans ...
   ✗ Failed: Only one cluster formed
→ Implementing Agglomerative_Single ...


/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


   ✓ Done | Label change: 68.88%
→ Implementing Agglomerative_Complete ...
   ✓ Done | Label change: 68.88%
→ Implementing Agglomerative_Average ...
   ✓ Done | Label change: 68.88%
→ Implementing Agglomerative_Ward ...
   ✓ Done | Label change: 68.88%
→ Implementing DBSCAN ...
   ✗ Failed: Only one cluster formed
→ Implementing OPTICS ...
   ✗ Failed: Only one cluster formed
→ Implementing HDBSCAN ...
   ✗ Failed: Only one cluster formed
→ Implementing MeanShift ...
   ✗ Failed: Only one cluster formed
→ Implementing GMM ...


...

/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.12/dist-packages/sklearn/mixture/_base.py:269: ConvergenceWarning: Best performing initialization did not converge. Try different init parameters, or increase max_iter, tol, or check for degenerate data.
  warnings.warn(


   ✗ Failed: Only one cluster formed
→ Implementing SpectralClustering ...
   ✓ Done | Label change: 69.24%
→ Implementing BIRCH ...
   ✗ Failed: Only one cluster formed
→ Implementing AffinityPropagation ...


/usr/local/lib/python3.12/dist-packages/sklearn/cluster/_birch.py:727: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (3). Decrease the threshold.
  warnings.warn(


   ✗ Failed: Only one cluster formed


/usr/local/lib/python3.12/dist-packages/sklearn/cluster/_affinity_propagation.py:50: UserWarning: All samples have mutually equal similarities. Returning arbitrary cluster center(s).
  warnings.warn(


In [ ]:
NUM_CLASSES = 3
MODEL_NAME = "MobileNetV3"

print(f"\n===== Processing {MODEL_NAME} =====")

X = np.load(f"{MODEL_NAME}_features.npy").astype(np.float64)

algorithms = get_algorithms(X)
results = []

for algo_name, algo in algorithms.items():
    print(f"→ Implementing {algo_name} ...")

    try:
        # Case 1: callable algorithms (Fuzzy, Spherical)
        if callable(algo):
            labels = algo(X)

        # Case 2: GMM / BayesianGMM
        elif hasattr(algo, "predict") and not hasattr(algo, "fit_predict"):
            algo.fit(X)
            labels = algo.predict(X)

        # Case 3: sklearn clustering
        else:
            labels = algo.fit_predict(X)

        # Skip degenerate clustering
        if len(np.unique(labels)) < 2:
            raise ValueError("Only one cluster formed")

        save_cluster_labels_csv(
            MODEL_NAME,
            algo_name,
            image_names,
            original_labels,
            labels
        )

        change = compute_label_change(original_labels_enc, labels)
        results.append([MODEL_NAME, algo_name, change])

        print(f"   ✓ Done | Label change: {change:.2f}%")

    except Exception as e:
        print(f"   ✗ Failed: {str(e)}")
        continue


pd.DataFrame(
    results,
    columns=["model", "clustering", "label_percentage_change"]
).to_csv(
    "clustering_results.csv",
    mode="a",
    header=not os.path.exists("clustering_results.csv"),
    index=False
)



===== Processing MobileNetV3 =====
→ Implementing KMeans ...
   ✓ Done | Label change: 57.69%
→ Implementing MiniBatchKMeans ...
   ✓ Done | Label change: 77.18%
→ Implementing KMedoids_PAM ...
   ✓ Done | Label change: 66.76%
→ Implementing BisectingKMeans ...
   ✓ Done | Label change: 59.10%
→ Implementing SphericalKMeans ...
   ✓ Done | Label change: 66.07%
→ Implementing FuzzyCMeans ...
   ✓ Done | Label change: 57.44%
→ Implementing Agglomerative_Single ...
   ✓ Done | Label change: 68.96%
→ Implementing Agglomerative_Complete ...
   ✓ Done | Label change: 58.09%
→ Implementing Agglomerative_Average ...
   ✓ Done | Label change: 68.75%
→ Implementing Agglomerative_Ward ...
   ✓ Done | Label change: 69.99%
→ Implementing DBSCAN ...
   ✗ Failed: Only one cluster formed
→ Implementing OPTICS ...
   ✓ Done | Label change: 99.85%
→ Implementing HDBSCAN ...
   ✓ Done | Label change: 69.38%
→ Implementing MeanShift ...
   ✓ Done | Label change: 70.16%
→ Implementing GMM ...
   ✗ Failed:

/usr/local/lib/python3.12/dist-packages/sklearn/mixture/_base.py:269: ConvergenceWarning: Best performing initialization did not converge. Try different init parameters, or increase max_iter, tol, or check for degenerate data.
  warnings.warn(


   ✓ Done | Label change: 69.45%
→ Implementing SpectralClustering ...
   ✓ Done | Label change: 58.05%
→ Implementing BIRCH ...
   ✗ Failed: Only one cluster formed
→ Implementing AffinityPropagation ...


/usr/local/lib/python3.12/dist-packages/sklearn/cluster/_birch.py:727: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (3). Decrease the threshold.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/cluster/_affinity_propagation.py:140: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


   ✓ Done | Label change: 99.98%


In [ ]:
NUM_CLASSES = 3
MODEL_NAME = "VGG19"

print(f"\n===== Processing {MODEL_NAME} =====")

X = np.load(f"{MODEL_NAME}_features.npy").astype(np.float64)

algorithms = get_algorithms(X)
results = []

for algo_name, algo in algorithms.items():
    print(f"→ Implementing {algo_name} ...")

    try:
        # Case 1: callable algorithms (Fuzzy, Spherical)
        if callable(algo):
            labels = algo(X)

        # Case 2: GMM / BayesianGMM
        elif hasattr(algo, "predict") and not hasattr(algo, "fit_predict"):
            algo.fit(X)
            labels = algo.predict(X)

        # Case 3: sklearn clustering
        else:
            labels = algo.fit_predict(X)

        # Skip degenerate clustering
        if len(np.unique(labels)) < 2:
            raise ValueError("Only one cluster formed")

        save_cluster_labels_csv(
            MODEL_NAME,
            algo_name,
            image_names,
            original_labels,
            labels
        )

        change = compute_label_change(original_labels_enc, labels)
        results.append([MODEL_NAME, algo_name, change])

        print(f"   ✓ Done | Label change: {change:.2f}%")

    except Exception as e:
        print(f"   ✗ Failed: {str(e)}")
        continue


pd.DataFrame(
    results,
    columns=["model", "clustering", "label_percentage_change"]
).to_csv(
    "clustering_results.csv",
    mode="a",
    header=not os.path.exists("clustering_results.csv"),
    index=False
)



===== Processing VGG19 =====
→ Implementing KMeans ...
   ✗ Failed: Only one cluster formed
→ Implementing MiniBatchKMeans ...
   ✗ Failed: Only one cluster formed
→ Implementing KMedoids_PAM ...


/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


   ✗ Failed: Only one cluster formed
→ Implementing BisectingKMeans ...
   ✗ Failed: Only one cluster formed
→ Implementing SphericalKMeans ...
   ✗ Failed: Only one cluster formed
→ Implementing FuzzyCMeans ...
   ✗ Failed: Only one cluster formed
→ Implementing Agglomerative_Single ...


/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


   ✓ Done | Label change: 68.88%
→ Implementing Agglomerative_Complete ...
   ✓ Done | Label change: 68.88%
→ Implementing Agglomerative_Average ...
   ✓ Done | Label change: 68.88%
→ Implementing Agglomerative_Ward ...
   ✓ Done | Label change: 68.88%
→ Implementing DBSCAN ...
   ✗ Failed: Only one cluster formed
→ Implementing OPTICS ...
   ✗ Failed: Only one cluster formed
→ Implementing HDBSCAN ...
   ✗ Failed: Only one cluster formed
→ Implementing MeanShift ...
   ✗ Failed: Only one cluster formed
→ Implementing GMM ...


...

/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.12/dist-packages/sklearn/mixture/_base.py:269: ConvergenceWarning: Best performing initialization did not converge. Try different init parameters, or increase max_iter, tol, or check for degenerate data.
  warnings.warn(


   ✗ Failed: Only one cluster formed
→ Implementing SpectralClustering ...
   ✓ Done | Label change: 69.24%
→ Implementing BIRCH ...
   ✗ Failed: Only one cluster formed
→ Implementing AffinityPropagation ...


/usr/local/lib/python3.12/dist-packages/sklearn/cluster/_birch.py:727: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (3). Decrease the threshold.
  warnings.warn(


   ✗ Failed: Only one cluster formed


/usr/local/lib/python3.12/dist-packages/sklearn/cluster/_affinity_propagation.py:50: UserWarning: All samples have mutually equal similarities. Returning arbitrary cluster center(s).
  warnings.warn(


In [ ]:
NUM_CLASSES = 3
MODEL_NAME = "Xception"

print(f"\n===== Processing {MODEL_NAME} =====")

X = np.load(f"{MODEL_NAME}_features.npy").astype(np.float64)

algorithms = get_algorithms(X)
results = []

for algo_name, algo in algorithms.items():
    print(f"→ Implementing {algo_name} ...")

    try:
        # Case 1: callable algorithms (Fuzzy, Spherical)
        if callable(algo):
            labels = algo(X)

        # Case 2: GMM / BayesianGMM
        elif hasattr(algo, "predict") and not hasattr(algo, "fit_predict"):
            algo.fit(X)
            labels = algo.predict(X)

        # Case 3: sklearn clustering
        else:
            labels = algo.fit_predict(X)

        # Skip degenerate clustering
        if len(np.unique(labels)) < 2:
            raise ValueError("Only one cluster formed")

        save_cluster_labels_csv(
            MODEL_NAME,
            algo_name,
            image_names,
            original_labels,
            labels
        )

        change = compute_label_change(original_labels_enc, labels)
        results.append([MODEL_NAME, algo_name, change])

        print(f"   ✓ Done | Label change: {change:.2f}%")

    except Exception as e:
        print(f"   ✗ Failed: {str(e)}")
        continue


pd.DataFrame(
    results,
    columns=["model", "clustering", "label_percentage_change"]
).to_csv(
    "clustering_results.csv",
    mode="a",
    header=not os.path.exists("clustering_results.csv"),
    index=False
)



===== Processing Xception =====
→ Implementing KMeans ...
   ✓ Done | Label change: 84.58%
→ Implementing MiniBatchKMeans ...
   ✓ Done | Label change: 65.63%
→ Implementing KMedoids_PAM ...
   ✓ Done | Label change: 48.39%
→ Implementing BisectingKMeans ...
   ✓ Done | Label change: 49.23%
→ Implementing SphericalKMeans ...
   ✓ Done | Label change: 85.62%
→ Implementing FuzzyCMeans ...
   ✓ Done | Label change: 81.37%
→ Implementing Agglomerative_Single ...
   ✓ Done | Label change: 68.92%
→ Implementing Agglomerative_Complete ...
   ✓ Done | Label change: 65.09%
→ Implementing Agglomerative_Average ...
   ✓ Done | Label change: 65.19%
→ Implementing Agglomerative_Ward ...
   ✓ Done | Label change: 46.46%
→ Implementing DBSCAN ...
   ✓ Done | Label change: 87.26%
→ Implementing OPTICS ...
   ✓ Done | Label change: 99.85%
→ Implementing HDBSCAN ...
   ✓ Done | Label change: 70.12%
→ Implementing MeanShift ...
   ✓ Done | Label change: 70.66%
→ Implementing GMM ...
   ✓ Done | Label c

/usr/local/lib/python3.12/dist-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
NUM_CLASSES = 3
MODEL_NAME = "Xception"

print(f"\n===== Processing {MODEL_NAME} =====")

X = np.load(f"{MODEL_NAME}_features.npy").astype(np.float64)

algorithms = get_algorithms(X)
results = []

for algo_name, algo in algorithms.items():
    print(f"→ Implementing {algo_name} ...")

    try:
        # Case 1: callable algorithms (Fuzzy, Spherical)
        if callable(algo):
            labels = algo(X)

        # Case 2: GMM / BayesianGMM
        elif hasattr(algo, "predict") and not hasattr(algo, "fit_predict"):
            algo.fit(X)
            labels = algo.predict(X)

        # Case 3: sklearn clustering
        else:
            labels = algo.fit_predict(X)

        # Skip degenerate clustering
        if len(np.unique(labels)) < 2:
            raise ValueError("Only one cluster formed")

        save_cluster_labels_csv(
            MODEL_NAME,
            algo_name,
            image_names,
            original_labels,
            labels
        )

        change = compute_label_change(original_labels_enc, labels)
        results.append([MODEL_NAME, algo_name, change])

        print(f"   ✓ Done | Label change: {change:.2f}%")

    except Exception as e:
        print(f"   ✗ Failed: {str(e)}")
        continue


pd.DataFrame(
    results,
    columns=["model", "clustering", "label_percentage_change"]
).to_csv(
    "clustering_results.csv",
    mode="a",
    header=not os.path.exists("clustering_results.csv"),
    index=False
)



===== Processing Xception =====
→ Implementing KMeans ...
   ✓ Done | Label change: 84.58%
→ Implementing MiniBatchKMeans ...
   ✓ Done | Label change: 65.63%
→ Implementing KMedoids_PAM ...
   ✓ Done | Label change: 48.39%
→ Implementing BisectingKMeans ...
   ✓ Done | Label change: 49.23%
→ Implementing SphericalKMeans ...
   ✓ Done | Label change: 85.62%
→ Implementing FuzzyCMeans ...
   ✓ Done | Label change: 81.24%
→ Implementing Agglomerative_Single ...
   ✓ Done | Label change: 68.92%
→ Implementing Agglomerative_Complete ...
   ✓ Done | Label change: 65.09%
→ Implementing Agglomerative_Average ...
   ✓ Done | Label change: 65.19%
→ Implementing Agglomerative_Ward ...
   ✓ Done | Label change: 46.46%
→ Implementing DBSCAN ...
   ✓ Done | Label change: 87.26%
→ Implementing OPTICS ...
   ✓ Done | Label change: 99.85%
→ Implementing HDBSCAN ...
   ✓ Done | Label change: 70.12%
→ Implementing MeanShift ...
   ✓ Done | Label change: 70.66%
→ Implementing GMM ...
   ✓ Done | Label c

/usr/local/lib/python3.12/dist-packages/sklearn/cluster/_affinity_propagation.py:140: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


In [ ]:
from google.colab import files

files.download("clustering_results.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r clustering_labels.zip clustering_labels
files.download("clustering_labels.zip")


updating: clustering_labels/ (stored 0%)
updating: clustering_labels/MobileNetV3_MeanShift.csv (deflated 88%)
updating: clustering_labels/VGG19_Agglomerative_Single.csv (deflated 90%)
updating: clustering_labels/VGG19_Agglomerative_Ward.csv (deflated 90%)
updating: clustering_labels/EfficientNetB0_OPTICS.csv (deflated 86%)
updating: clustering_labels/MobileNetV3_HDBSCAN.csv (deflated 90%)
updating: clustering_labels/EfficientNetB0_AffinityPropagation.csv (deflated 86%)
updating: clustering_labels/EfficientNetB0_MeanShift.csv (deflated 88%)
updating: clustering_labels/EfficientNetB0_BisectingKMeans.csv (deflated 88%)
updating: clustering_labels/MobileNetV3_Agglomerative_Ward.csv (deflated 88%)
updating: clustering_labels/EfficientNetB0_Agglomerative_Average.csv (deflated 88%)
updating: clustering_labels/EfficientNetB0_MiniBatchKMeans.csv (deflated 88%)
updating: clustering_labels/Xception_Agglomerative_Single.csv (deflated 90%)
updating: clustering_labels/ResNet152_Agglomerative_Ward.cs

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import silhouette_score

# ============================
# LOAD FEATURES
# ============================

X_eff = np.load("EfficientNetB0_features.npy")
X_xcep = np.load("Xception_features.npy")

# ============================
# LOAD CLUSTER LABEL FILES
# ============================

eff_kmeans_df = pd.read_csv("clustering_labels/EfficientNetB0_KMeans.csv")
xcep_birch_df = pd.read_csv("clustering_labels/Xception_BIRCH.csv")

labels_eff = eff_kmeans_df["cluster_label"].values
labels_xcep = xcep_birch_df["cluster_label"].values

# ============================
# REMOVE NOISE IF PRESENT (-1)
# ============================

def compute_silhouette(X, labels, name):
    unique_labels = np.unique(labels)

    if len(unique_labels) < 2:
        print(f"{name}: Cannot compute silhouette (only one cluster)")
        return None

    # Remove noise label -1 if exists
    mask = labels != -1
    X_clean = X[mask]
    labels_clean = labels[mask]

    if len(np.unique(labels_clean)) < 2:
        print(f"{name}: Not enough valid clusters after removing noise")
        return None

    score = silhouette_score(X_clean, labels_clean)
    print(f"{name} Silhouette Score: {score:.4f}")
    return score


# ============================
# COMPUTE SCORES
# ============================

score_min = compute_silhouette(
    X_eff,
    labels_eff,
    "Minimum Change (EfficientNetB0 + KMeans)"
)

score_max = compute_silhouette(
    X_xcep,
    labels_xcep,
    "Maximum Change (Xception + Birch)"
)


Minimum Change (EfficientNetB0 + KMeans) Silhouette Score: 0.8246
Maximum Change (Xception + Birch) Silhouette Score: 0.7279
